In [1]:
import pandas as pd
import numpy as np

In [2]:
# pd.set_option('display.max_rows', 600)
# pd.set_option('display.max_columns', 600)

In [3]:
df1 = pd.read_excel('../Central Branch  trackers/Central Division 2021 Debris Removal Tracker.xlsx')
df2 = pd.read_csv('Power BI CSVs/BIO.csv')
df3 = pd.read_csv('Power BI CSVs/Arch.csv')

In [4]:
smartsheets = df1.copy()
bio = df2.copy()
arch = df3.copy()

# smart sheet columns needed check:
APN_ROW Segment 'Arch Monitor Needed','Bio Monitor Needed'

In [5]:
# select columns we need
smartsheets = smartsheets[['APN_ROW Segment','Arch Monitor Needed','Arch Assessment', 'Bio Monitor Needed','Bio Assessment']]

In [6]:
smartsheets['Arch Assessment'] = pd.to_datetime(smartsheets['Arch Assessment']).dt.date
smartsheets['Bio Assessment'] = pd.to_datetime(smartsheets['Bio Assessment']).dt.date

In [7]:
smartsheets.rename(columns={'Arch Monitor Needed': 'Arch Monitor Needed_SS','Arch Assessment':'Arch Assessment_SS',
                            'Bio Monitor Needed':'Bio Monitor Needed_SS','Bio Assessment':'Bio Assessment_SS'}, inplace=True)

# BIO columns needed:
APN , Biological monitoring recommended?

In [8]:
# bio['Biological monitoring recommended?'].value_counts()

In [9]:
# select columns we need
bio = bio[['APN', 'Biological monitoring recommended?','County', 'Observation Date']]

In [10]:
bio = bio[bio['County'].isin(['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER'])]

In [11]:
bio['Observation Date'] = pd.to_datetime(bio['Observation Date']).dt.date

In [12]:
# change column name values to match smart sheets
bio['Biological monitoring recommended?'] = bio['Biological monitoring recommended?'].str.replace('No Debris, No Tree', 'No Bio Monitoring Needed for Debris or Trees')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('Yes Debris, Yes Tree', 'Bio Monitoring Needed for Debris AND Trees')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('No Debris, Yes Tree', 'Bio Monitoring Needed for Trees, not Debris')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('Yes Debris, No Tree','Bio Monitoring Needed for Debris, not Trees')

In [13]:
bio.rename(columns={'Observation Date':'Observation Date_BIO'}, inplace=True)

# ARCH columns needed:
APN , Archaeological Resources Present?

In [14]:
# select columns we need
arch = arch[['APN', 'Monitoring Recommendation','Date', 'County']]

In [15]:
arch = arch[arch['County'].isin(['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER'])]

In [16]:
arch['Date'] = pd.to_datetime(arch['Date']).dt.date

In [17]:
# change column name values to match smart sheets
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('No Debris, No Tree', 'No Arch Monitoring Needed for Debris or Trees')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('Yes Debris, Yes Tree', 'Arch Monitoring Needed for Debris AND Trees')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('No Debris, Yes Tree', 'Arch Monitoring Needed for Trees, not Debris')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('Yes Debris, No Tree', 'Arch Monitoring Needed for Debris, not Trees')

In [18]:
arch.rename(columns={'Monitoring Recommendation':'Monitoring Recommendation_ARCH', 'Date': 'Date_ARCH'},inplace=True)

# Merge Data bases
* smartsheets and BIO
* smasrtsheet and Arch

# Smartsheet and BIO

In [19]:
# Merge Smartsheets and BIO

smartsheets_bio = smartsheets.merge(bio,
                          left_on='APN_ROW Segment',
                          right_on='APN',
                          how='outer')

In [20]:
smartsheets_bio.rename(columns={'APN':'APN_BIO'}, inplace=True)

In [21]:
smartsheets_bio.head(0)

,APN_ROW Segment,Arch Monitor Needed_SS,Arch Assessment_SS,Bio Monitor Needed_SS,Bio Assessment_SS,APN_BIO,Biological monitoring recommended?,County,Observation Date_BIO


In [22]:
smartsheets_bio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687 entries, 0 to 686
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   APN_ROW Segment                     686 non-null    object
 1   Arch Monitor Needed_SS              333 non-null    object
 2   Arch Assessment_SS                  332 non-null    object
 3   Bio Monitor Needed_SS               397 non-null    object
 4   Bio Assessment_SS                   397 non-null    object
 5   APN_BIO                             402 non-null    object
 6   Biological monitoring recommended?  402 non-null    object
 7   County                              402 non-null    object
 8   Observation Date_BIO                402 non-null    object
dtypes: object(9)
memory usage: 53.7+ KB


In [23]:
smartsheets_bio['Bio Monitor Needed_SS'] = smartsheets_bio['Bio Monitor Needed_SS'].fillna("0")
smartsheets_bio['Biological monitoring recommended?'] = smartsheets_bio['Biological monitoring recommended?'].fillna('0')
smartsheets_bio['Observation Date_BIO'] = smartsheets_bio['Observation Date_BIO'].fillna("0")
smartsheets_bio['Bio Assessment_SS'] = smartsheets_bio['Bio Assessment_SS'].fillna("0")

# Smartsheet and Arch

In [24]:
smartsheets_ARCH = smartsheets.merge(arch,
                    left_on='APN_ROW Segment',
                    right_on='APN',
                    how='outer')

In [25]:
smartsheets_ARCH.rename(columns={'APN':'APN_ARCH'}, inplace=True)

In [26]:
smartsheets_ARCH.head(0)

,APN_ROW Segment,Arch Monitor Needed_SS,Arch Assessment_SS,Bio Monitor Needed_SS,Bio Assessment_SS,APN_ARCH,Monitoring Recommendation_ARCH,Date_ARCH,County


In [27]:
smartsheets_ARCH['Date_ARCH'].dtypes

dtype('O')

In [28]:
smartsheets_ARCH['Arch Monitor Needed_SS'] = smartsheets_ARCH['Arch Monitor Needed_SS'].fillna("0")
smartsheets_ARCH['Monitoring Recommendation_ARCH'] = smartsheets_ARCH['Monitoring Recommendation_ARCH'].fillna('0')
smartsheets_ARCH['Date_ARCH'] = smartsheets_ARCH['Date_ARCH'].fillna("0")
smartsheets_ARCH['Arch Assessment_SS'] = smartsheets_ARCH['Arch Assessment_SS'].fillna("0")

# Set up column order for each Data base

In [29]:
# ARch and smart sheets
smartsheets_ARCH = smartsheets_ARCH[['APN_ROW Segment', 'APN_ARCH',
       'Arch Monitor Needed_SS', 'Monitoring Recommendation_ARCH',
                                     'Arch Assessment_SS','Date_ARCH']]

In [30]:
# BIO and smasrt sheets
smartsheets_bio = smartsheets_bio[['APN_ROW Segment', 'APN_BIO',
                                   'Bio Monitor Needed_SS', 'Biological monitoring recommended?',
                                   'Bio Assessment_SS','Observation Date_BIO']]

# Compare columns in smart sheet to other databases

In [31]:
def compare_cols(df, comp1, comp2):
    if df[comp1] == df[comp2]:
        return 1
    else:
        return 0

# compart ARCH monitor needed to smartsheet

In [32]:
smartsheets_ARCH.insert(smartsheets_ARCH.columns.get_loc('Monitoring Recommendation_ARCH') + 1, "ARC Check",
            smartsheets_ARCH.apply(compare_cols, comp1='Arch Monitor Needed_SS', comp2='Monitoring Recommendation_ARCH',axis=1))

# comapre dates for ARCH and smartsheets dates

In [33]:
smartsheets_ARCH.insert(smartsheets_ARCH.columns.get_loc('Date_ARCH') + 1, 'Check Arch Dates',
            smartsheets_ARCH.apply(compare_cols, comp1='Arch Assessment_SS', comp2='Date_ARCH', axis=1))

# check BIO and Smartsheet dates

In [34]:
smartsheets_bio.insert(smartsheets_bio.columns.get_loc('Observation Date_BIO') + 1, 'Check BIO Dates',
            smartsheets_bio.apply(compare_cols, comp1='Bio Assessment_SS', comp2='Observation Date_BIO',axis=1))

# Compare BIO monitor needed to  smartsheets 

In [35]:
smartsheets_bio.insert(smartsheets_bio.columns.get_loc('Biological monitoring recommended?') + 1, 'BIO Check',
                                  smartsheets_bio.apply(compare_cols, comp1='Bio Monitor Needed_SS',
                                              comp2='Biological monitoring recommended?',axis=1))

# highligh the columns that are comparing values

In [36]:
df_style_BIO = smartsheets_bio.style.set_properties(**{'background-color': '#ffffb3'},
                          subset=['BIO Check', 'Check BIO Dates'])

In [37]:
df_style_ARCH = smartsheets_ARCH.style.set_properties(**{'background-color': '#ffffb3'},
                                                    subset=["ARC Check", 'Check Arch Dates'])

In [38]:
df_style_BIO.to_excel('BIO checks.xlsx')

In [39]:
df_style_ARCH.to_excel('ARCH check.xlsx')